In [1]:
import os
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import argparse
import logging
from datetime import datetime, timedelta, date
import torch
import torch.nn.functional as F
import sklearn.metrics as metrics
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch_geometric.utils.convert import from_networkx

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Data Loading
data_root_path = '/media/usr/HDD/Data/NAVER'
partition_list = os.listdir(data_root_path)
partition_list = [p for p in partition_list if p[0]=='2']
partition_list = np.sort(partition_list)

data_path = '/media/usr/HDD/Working/Naver_Data/data_parsing'

sids_all = []
eventID_all = []

for partition in partition_list:
    try: 
        eventID_list = [filename.split('.')[0] for filename in os.listdir(os.path.join(data_path, 'networks', partition)) if filename[0] != '.']
        eventID_list = np.unique(eventID_list)
        eventID_all.append(eventID_list)

        for eventID in eventID_list:
            with open(os.path.join(data_path, 'networks', partition, '{}.pickle'.format(eventID)), 'rb') as f:
                accident_info = pickle.load(f)
            G = nx.read_gpickle(os.path.join(data_path, 'networks', partition, '{}.gpickle'.format(eventID)))

            sids_all.append(accident_info[1])
            sids_all.append(accident_info[2])
    except:
        continue

eventID_all = [x for y in eventID_all for x in y]
eventID_all = np.unique(eventID_all)
        
sids_all = [x for y in sids_all for x in y]
sids_all = np.unique(sids_all)

print('# of all Events, # of sids = ', len(eventID_all), len(sids_all))

data_extraction_path = '/media/usr/HDD/Data/NAVER_df'
# filtered_ID = [eventID for eventID in eventID_all if eventID in os.listdir(data_extraction_path)]

## load accident_all
accident_all = pd.read_csv('./data/accident_all.csv', index_col=0)
print("# of filtered Events = ", len(accident_all))


# Profile Extraction Functions
def profile_extraction2(speed_all):
    speed_all2 = speed_all.copy()
    # Day of Week => monday : 0, sunday : 6
    speed_all2['weekday'] = [s.weekday() for s in speed_all.index]
    speed_all2['timestamp'] = [s.time() for s in speed_all.index]
    
    profile_mean = speed_all2.groupby(['weekday', 'timestamp']).mean()
    profile_std = speed_all2.groupby(['weekday', 'timestamp']).std()
    
    speed_all2 = speed_all2.drop(['weekday', 'timestamp'], axis=1)
    
    return speed_all2, profile_mean, profile_std

# of all Events, # of sids =  4203 15629
# of filtered Events =  849


In [4]:
accident_all['accident_sid'].value_counts()

1210005301    20
1030001902    19
1220005401    15
1210003000    13
1130052300    12
              ..
2210066900     1
1120013402     1
1100005401     1
1070010402     1
1040020100     1
Name: accident_sid, Length: 482, dtype: int64

In [5]:
target_sid = 1210005301     ## 1030001902    ## 1210005301  ## 1210003000
accident_case = accident_all[accident_all.loc[:, 'accident_sid'] == target_sid]
eventID = accident_case.eventId.iloc[0]

## Incident Region Map images

In [7]:
import folium

In [31]:
    target_sid = 1210005301     ## 1030001902    ## 1210005301
    accident_case = accident_all[accident_all.loc[:, 'accident_sid'] == target_sid]
    eventID = accident_case.eventId.iloc[0]
    normalize = 'standard'

    eventID = str(eventID)

    # accident info : 0 : description / 1 : sid / 2 : sid 
    # what sids?
    with open(os.path.join(data_path, 'speeds', eventID, '{}.pickle'.format(eventID)), 'rb') as f:
        accident_info = pickle.load(f)
    G = nx.read_gpickle(os.path.join(data_path, 'speeds', eventID, '{}.gpickle'.format(eventID)))

    sid_list = accident_info[1] + accident_info[2]

    accident_sid = accident_info[0]['sids'][0]
    accident_created = accident_info[0]['created']

    # feature extraction
    with open(os.path.join(data_extraction_path, eventID), 'rb') as f:
        test = pickle.load(f)
    speed_inflow = test['speed_inflow']
    speed_outflow = test['speed_outflow']

    speed_all = pd.concat([speed_inflow, speed_outflow], axis=1)
    speed_all = speed_all.dropna(axis=1, how='all')
    
    tmp = speed_all[accident_sid].iloc[:, 0].values
    speed_all = speed_all.drop([accident_sid], axis=1)
    speed_all[accident_sid] = tmp

    ## selected nodes
    sid_list = list(set(list(speed_inflow.columns) + list(speed_outflow.columns) + [accident_sid]))
    H = nx.subgraph(G, sid_list)




In [30]:
lat = accident_case.lat.iloc[0]
lng = accident_case.lng.iloc[0]
M = folium.Map((lat, lng), zoom_start=15)

# icon_star = plugins.BeautifyIcon(
#     icon='star',
#     inner_icon_style='color:red;font-size:20px;',
#     background_color='transparent',
#     border_color='transparent',
# )

# for eventID in selected_ID:
eventID = str(eventID)
accident_sid = target_sid

# with open(os.path.join(data_path, 'speeds', eventID, '{}.pickle'.format(eventID)), 'rb') as f:
#     accident_info = pickle.load(f)
    
# folium.Circle((accident_info[0]['lat'], accident_info[0]['lng']),
#             radius = 100,
#             color = 'black',
#             opacity = 0.8,
#             fill_color = 'black').add_to(M)


H = nx.subgraph(G, np.unique(path_inflow))


folium.Marker(
    [H.nodes('pos')[accident_sid][1], H.nodes('pos')[accident_sid][0]], 
    tooltip='incident'#, icon=icon_star
).add_to(M)



for e in H.edges:
    start = H.nodes('pos')[e[0]][::-1]
    end = H.nodes('pos')[e[1]][::-1]
    
    if e[0] in np.array(path_inflow)[:,1]:
        cc = 'red'
    elif e[0] in np.array(path_inflow)[:,2]:
        cc = 'blue'
    elif e[0] in np.array(path_inflow)[:,3]:
        cc = 'green'
    
    folium.PolyLine(
        locations=[start, end],
        color=cc, weight=5
    ).add_to(M)

H = nx.subgraph(G, np.unique(path_outflow))

for e in H.edges:
    start = H.nodes('pos')[e[0]][::-1]
    end = H.nodes('pos')[e[1]][::-1]
    
    if e[0] in np.array(path_outflow)[:,1]:
        cc = 'green'
    elif e[0] in np.array(path_outflow)[:,2]:
        cc = 'blue'
    elif e[0] in np.array(path_outflow)[:,3]:
        cc = 'red'
    
    folium.PolyLine(
        locations=[start, end],
        color=cc, weight=5
    ).add_to(M)



M

[eventId                                   3685247
 created                       2020-09-27 20:04:00
 startExpect                   2020-09-27 20:02:00
 endExpect                     2020-09-27 20:22:00
 start                                        None
 end                                           NaT
 cp                                       tbs 교통방송
 category                                 ACCIDENT
 description    성산대교 (남단 → 북단) 북단부근 1,2차로 추돌사고로 정체
 sids                                 [1130052300]
 lng                                    126.893413
 lat                                     37.555745
 Name: 40, dtype: object,
 [1130052300, 1130052300, 1130052801, 1130056200, 1130057600],
 [1130052300,
  1130052300,
  1180003006,
  1180004000,
  1180115200,
  1180115300,
  1180116000,
  1180116400,
  1180116600,
  1180117000,
  1180117500]]

In [54]:
M = folium.Map((accident_all.lat.mean(), accident_all.lng.mean()), zoom_start=12.3)

# # 배경지도 타일 설정하기
# tiles = "CartoDB positron"
# # 배경지도 타일 레이어를 지도에 추가하기
# folium.TileLayer(tiles=tiles).add_to(M)


for target_sid in [1210005301, 1030001902, 1220005401, 1210003000, 1130052300 ]:
    # target_sid = 1210005301     ## ['1210005301', '1030001902', '1220005401', '1210003000', '1130052300']
    accident_case = accident_all[accident_all.loc[:, 'accident_sid'] == target_sid]
    eventID = accident_case.eventId.iloc[0]
    normalize = 'standard'

    eventID = str(eventID)

    # accident info : 0 : description / 1 : sid / 2 : sid 
    # what sids?
    with open(os.path.join(data_path, 'speeds', eventID, '{}.pickle'.format(eventID)), 'rb') as f:
        accident_info = pickle.load(f)
    G = nx.read_gpickle(os.path.join(data_path, 'speeds', eventID, '{}.gpickle'.format(eventID)))

    sid_list = accident_info[1] + accident_info[2]

    accident_sid = accident_info[0]['sids'][0]
    accident_created = accident_info[0]['created']

    # feature extraction
    with open(os.path.join(data_extraction_path, eventID), 'rb') as f:
        test = pickle.load(f)

    path_inflow = test['path_inflow']
    path_outflow = test['path_outflow']

    speed_inflow = test['speed_inflow']
    speed_outflow = test['speed_outflow']

    speed_all = pd.concat([speed_inflow, speed_outflow], axis=1)
    speed_all = speed_all.dropna(axis=1, how='all')
    
    tmp = speed_all[accident_sid].iloc[:, 0].values
    speed_all = speed_all.drop([accident_sid], axis=1)
    speed_all[accident_sid] = tmp

    ## selected nodes
    sid_list = list(set(list(speed_inflow.columns) + list(speed_outflow.columns) + [accident_sid]))
    H = nx.subgraph(G, sid_list)


    lat = accident_case.lat.iloc[0]
    lng = accident_case.lng.iloc[0]

    
    folium.Marker(
        [H.nodes('pos')[target_sid][1], H.nodes('pos')[target_sid][0]], 
        tooltip='incident', icon=folium.Icon(color='blue', icon='star')
    ).add_to(M)

    
    for e in H.edges:
        start = H.nodes('pos')[e[0]][::-1]
        end = H.nodes('pos')[e[1]][::-1]
        
        if e[0] in np.array(path_inflow)[:,1]:
            cc = 'red'
        elif e[0] in np.array(path_inflow)[:,2]:
            cc = 'blue'
        elif e[0] in np.array(path_inflow)[:,3]:
            cc = 'green'
        
        folium.PolyLine(
            locations=[start, end],
            color=cc, weight=4
        ).add_to(M)

    H = nx.subgraph(G, np.unique(path_outflow))

    for e in H.edges:
        start = H.nodes('pos')[e[0]][::-1]
        end = H.nodes('pos')[e[1]][::-1]
        
        if e[0] in np.array(path_outflow)[:,1]:
            cc = 'green'
        elif e[0] in np.array(path_outflow)[:,2]:
            cc = 'blue'
        elif e[0] in np.array(path_outflow)[:,3]:
            cc = 'red'
        
        folium.PolyLine(
            locations=[start, end],
            color=cc, weight=4
        ).add_to(M)


M